#### Lesson 3: Reflection and Blogpost Writing

#### Setup

In [1]:
#r "nuget:AutoGen"

using AutoGen.Core;
using AutoGen.OpenAI;
using AutoGen.OpenAI.Extension;
using Azure.AI.OpenAI;
using System.Threading;

var openAIKey = Environment.GetEnvironmentVariable("OPENAI_API_KEY") ?? throw new Exception("Please set the OPENAI_API_KEY environment variable.");
var openAIModel = "gpt-4o-mini";
var openaiClient = new OpenAIClient(openAIKey);


Installed Packages AutoGen, 0.0.17

The Task!

In [2]:
var task = """
    Write a concise but engaging blogpost about
    DeepLearning.AI. Make sure the blogpost is
    within 100 words.
    """;

Create a writer agent

In [3]:
var writer = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Writer",
    modelName: openAIModel,
    systemMessage: """
    You are a writer. You write engaging and concise blogpost (with title) on given topics.
    You must polish your writing based on the feedback you receive and give a refined version.
    Only return your final work without additional comments."
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

    var reply = await writer.SendAsync(task);

from: Writer
**Unlocking the Future with DeepLearning.AI**

DeepLearning.AI is at the forefront of artificial intelligence education, empowering individuals to harness the power of deep learning. Founded by AI pioneer Andrew Ng, the platform offers specialized courses designed to demystify complex concepts. With a hands-on approach, learners engage with real-world projects, transforming theoretical knowledge into practical skills. From beginners to seasoned professionals, DeepLearning.AI fosters a vibrant community where innovation thrives. By bridging the gap between academia and industry, it's paving the way for the next generation of AI leaders. Dive in and unlock your potential in the ever-evolving world of AI!



Adding reflection by creating a critic agent to reflect on the work of the writer agent.

In [4]:
var critic = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Critic",
    modelName: openAIModel,
    systemMessage: """
    You are a critic. You review the work of the writer and provide constructive feedback to help improve the quality of the content."
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

var conversation = await critic.SendAsync(
    receiver: writer,
    message: task,
    maxRound: 3);

// Use the last message in the conversation as the final result
var res = conversation.Last();

from: Writer
**Unlocking the Future with DeepLearning.AI**

DeepLearning.AI is at the forefront of artificial intelligence education, empowering individuals and organizations to harness the potential of deep learning. Founded by AI pioneer Andrew Ng, the platform offers comprehensive courses that demystify complex concepts, making them accessible to all. From beginner to advanced levels, learners gain hands-on experience through practical projects and real-world applications. With a vibrant community and up-to-date resources, DeepLearning.AI is not just an educational hub but a launchpad for innovation in AI. Whether you're looking to boost your career or explore AI's vast possibilities, DeepLearning.AI is your gateway to the future.

from: Critic
This blog post effectively highlights the key aspects of DeepLearning.AI, such as its educational focus, founder, and community engagement. However, there are a few areas for improvement to enhance clarity and engagement:

1. **Hook**: Consid

#### Nested chat using middleware pattern
Python AutoGen allows you to create a Json object task list and pass it to agent using `register_nested_chats`
This is not supported in AutoGen.Net.
But you can achieve the same using the middleware pattern.

In [5]:
var SEOReviewer = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "SEO_Reviewer",
    modelName: openAIModel,
    systemMessage: """
    You are an SEO reviewer, known for your ability to optimize content for search engines, ensuring that it ranks well and attracts organic traffic.
    Make sure your suggestion is concise (within 3 bullet points), concrete and to the point.
    Begin the review by stating your role.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [6]:
var LegalReviewer = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Legal_Reviewer",
    modelName: openAIModel,
    systemMessage: """
    You are a legal reviewer, known for your ability to ensure that content is legally compliant and free from any potential legal issues.
    Make sure your suggestion is concise (within 3 bullet points), concrete and to the point.
    Begin the review by stating your role.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [7]:
var EthicsReviewer = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Ethics_Reviewer",
    modelName: openAIModel,
    systemMessage: """
    You are an ethics reviewer, known for your ability to ensure that content is ethically sound and free from any potential ethical issues.
    Make sure your suggestion is concise (within 3 bullet points), concrete and to the point.
    Begin the review by stating your role.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [8]:
var MetaReviewer = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Meta_Reviewer",
    modelName: openAIModel,
    systemMessage: """
    You are a meta reviewer, you aggragate and review the work of other reviewers and give a final suggestion on the content."
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

Define nest chat middleware

In [9]:
class NestChatMiddleware : IMiddleware
{
    private readonly IAgent seo;
    private readonly IAgent legal;
    private readonly IAgent ethics;
    private readonly IAgent meta;
    private readonly IAgent writer;

    public NestChatMiddleware(IAgent writer, IAgent seo, IAgent legal, IAgent ethics, IAgent meta)
    {
        this.writer = writer;
        this.seo = seo;
        this.legal = legal;
        this.ethics = ethics;
        this.meta = meta;
    }

    public string? Name => nameof(NestChatMiddleware);

    public async Task<IMessage> InvokeAsync(MiddlewareContext context, IAgent critic, CancellationToken cancellationToken = default)
    {
        // trigger only when the last message is from writer
        if (context.Messages.Last().From != writer.Name)
        {
            return await critic.GenerateReplyAsync(context.Messages, context.Options, cancellationToken);
        }

        var messageToReview = context.Messages.Last();
        var reviewPrompt = $"""
            Review the following content.

            {messageToReview.GetContent()}
            """;
        // SEO Review
        var seoReview = await critic.SendAsync(
            receiver: seo,
            message: reviewPrompt,
            maxRound: 1);
        var legalReview = await critic.SendAsync(
            receiver: legal,
            message: reviewPrompt,
            maxRound: 1);

        var ethicsReview = await critic.SendAsync(
            receiver: ethics,
            message: reviewPrompt,
            maxRound: 1);

        var reviews = seoReview.Concat(legalReview).Concat(ethicsReview);

        var metaReview = await critic.SendAsync(
            receiver: meta,
            message: "Aggregrate feedback from all reviewers and give final suggestions on the writing.",
            chatHistory: reviews,
            maxRound: 1);

        var lastReview = metaReview.Last();
        lastReview.From = critic.Name;

        return lastReview;
    }
}

In [10]:
// Orchestrate the nested chats to solve the task
var middleware = new NestChatMiddleware(
    writer: writer,
    seo: SEOReviewer,
    legal: LegalReviewer,
    ethics: EthicsReviewer,
    meta: MetaReviewer);

In [11]:
var nestChatCritic = critic
    .RegisterMiddleware(middleware)
    .RegisterPrintMessage();

In [12]:
conversation = await nestChatCritic.SendAsync(
    message: task,
    receiver: writer,
    maxRound: 3);

var finalResult = conversation.Last();

from: Writer
**Unlocking AI Potential with DeepLearning.AI**

DeepLearning.AI stands at the forefront of artificial intelligence education, empowering individuals and businesses with cutting-edge knowledge. Founded by AI pioneer Andrew Ng, it offers a range of courses tailored for all skill levels, from beginners to advanced practitioners. With hands-on projects and real-world applications, students gain practical experience in machine learning and deep learning techniques. The platform fosters a vibrant community, encouraging collaboration and innovation. Whether you're looking to upskill or dive into AI for the first time, DeepLearning.AI provides the tools and resources needed to thrive in this rapidly evolving field. Join the AI revolution today!

from: SEO_Reviewer
As an SEO reviewer, here are my suggestions to optimize the content for search engines:

- **Incorporate Keywords**: Include specific, high-traffic keywords related to "AI education," "machine learning courses," and "de